## RUN ONCE ~ CREATE VIRTUAL ENV

In [ ]:
from google.colab import drive
from os.path import join
import sys

def mount_drive(ROOT):
    drive.mount(ROOT, force_remount=True)

ROOT = '/content/drive'
mount_drive(ROOT)

MY_GOOGLE_DRIVE_PATH = 'MyDrive/RL_NLP/PPO_NEW/NLP_RL_Docker_Version'
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print("PROJECT_PATH: ", PROJECT_PATH)
%cd "{PROJECT_PATH}"
sys.path.insert(0, PROJECT_PATH)

In [ ]:
!pip3 install virtualenv
!virtualenv myenv
!source myenv/bin/activate

In [ ]:
!pip install -r "{PROJECT_PATH}/metadata/requirements.txt"
!pip install -r "{PROJECT_PATH}/metadata/requirements1.txt"

In [ ]:
!pip install -e .

In [ ]:
!deactivate

## RUN FROM HERE.

In [ ]:
from google.colab import drive
from os.path import join
import sys

def mount_drive(ROOT):
    drive.mount(ROOT, force_remount=True)

ROOT = '/content/drive'
mount_drive(ROOT)

MY_GOOGLE_DRIVE_PATH = 'MyDrive/RL_NLP/PPO_NEW/NLP_RL_Docker_Version'
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)
print("PROJECT_PATH: ", PROJECT_PATH)
%cd "{PROJECT_PATH}"
sys.path.insert(0, PROJECT_PATH)

In [ ]:
# Doesn't work because collab uses different shell than virtualenv !source myenv/bin/activate
!pip install transformers
!mkdir results

## !!IMPORTANT
> Ignore all cell before this, 
> if you are running from DOCKER in your local machine, run
> the cell below. Don't run the cell below otherwise.


In [2]:
%cd /NLP_RL_Docker_Version
!pip install -e .

/NLP_RL_Docker_Version
Obtaining file:///NLP_RL_Docker_Version
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      error: Multiple top-level packages discovered in a flat-layout: ['results', 'ENV_UTIL', 'metadata', 'transfer_task'].
      
      To avoid accidental inclusion of unwanted files or directories,
      setuptools will not proceed with this build.
      
      If you are trying to create a single distribution with multiple packages
      on purpose, you should not rely on automatic discovery.
      Instead, consider the following options:
      
      1. set up custom discovery (`find` directive with `include` or `exclude`)
      2. use a `src-layout`
      3. explicitly set `py_modules` or `packages` with a list of names
      
      To find more information, look for "package discovery" on setuptools docs.
      [end of output]
  
  note: 

In [3]:
!pip install transformers
!mkdir results


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
mkdir: cannot create directory ‘results’: File exists


In [4]:
import numpy as np
import pandas as pd
import time
import utils as U
import model as M
import model_nlp as MNLP
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import json
import copy
import gym
import sys
import os
sys.path.append(f'{os.getcwd()}/gym-examples')
import gym_examples
import numpy as np
import json


suffix = [['easy','medium','hard','naive'], \
          ['fnlp_easy','fnlp_medium','fnlp_hard','fnlp_naive'], \
          ['onlp_easy','onlp_medium','onlp_hard','onlp_naive'], \
          ['anlp_easy','anlp_medium','anlp_hard','anlp_naive']]

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def plot_ppo(name):
    arr=[]
    with open(f'results/test_reward_list_{name}.json', 'r') as file:
        arr=json.load(file)
        x,y=[arr[i][0] for i in range(len(arr))], [arr[i][1] for i in range(len(arr))]
        plt.plot(x,y)
        plt.xlabel(f'Frame Number')
        plt.ylabel(f'Cumulative Reward')
        plt.show()

In [ ]:
def run_best_env(number,name):
    str_arr = name.split('_')
    opt = 0
    if 'fnlp' in str_arr:
        opt = 1
    def policy(S):
        nonlocal opt
        if opt == 0:
            dist, value = model(S['visual'])
        elif opt == 1:
            dist, value = model(S['visual'],S['text'])
        action = dist.sample()
        return action.cpu().numpy().item()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if opt == 0:
        model = M.NNModel().to(device)
    elif opt == 1:
        model = MNLP.NNModelNLP().to(device)
    model.load_state_dict(torch.load(f'results/model_{name}.ml'))
    dbg=True
    episodes=1
    env = gym.make('gym_examples/RlNlpWorld-v0',render_mode="rgb_array")
    for _ in range(episodes):
        cumulative_reward,steps=0,0
        observation = env.reset(set_no=number,seed=42)
        state = copy.deepcopy(observation)
        observation['state'] = U.pre_process(observation)
        if opt == 1:
            observation['text'] = U.pre_process_text(model,observation)
        while True:
            print(observation['text'])
            if dbg==True:
                plt.imshow(state['visual'])
                plt.show()
            action = policy(observation)  # User-defined policy function
            observation, reward, terminated, info = env.step(action)
            print(f'Action {action}')
            state = copy.deepcopy(observation)
            observation['state'] = U.pre_process(observation)
            if opt == 1:
                observation['text'] = U.pre_process_text(model,observation)
            cumulative_reward += reward
            steps += 1
            if terminated: break
        print(f'Cumulative Reward ~ {cumulative_reward}; TimeTaken ~ {steps}')
    env.close()

### Best Agent

In [ ]:
from transformers import BertTokenizer 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def best_env(number=1):
    f,s,t = number//100,(number%100)//10,number%10
    exp_action = [0,3]*f + [1,4]*s + [2,5]*t
    exp_action_indx = 0
    def human_policy(observation):
        nonlocal exp_action_indx
        action=int(input())
        # try:
        #     action = exp_action[exp_action_indx]
        #     exp_action_indx += 1
        # except Exception as e:
        #     action = -1
        return action
    dbg=True
    episodes=1
    action, reward = None, None
    env = gym.make('gym_examples/RlNlpWorld-v0',render_mode="rgb_array", instr_type="policy")
    for _ in range(episodes):
        cumulative_reward,steps=0,0
        observation = env.reset(set_no=number)
        cnt,mx_iter=0,1002
        while steps<mx_iter:
            token = tokenizer(observation['text'])['input_ids']
            print(f"ACTION[{action}] REWARD[{reward}]\n" +\
                  f"{observation['text']}\n {token}\n" +\
                  f"{tokenizer.convert_ids_to_tokens(token)}\n" +\
                  f"{len(tokenizer.convert_ids_to_tokens(token))}"
                  )
            if dbg==True:
                plt.imshow(observation['visual'])
                plt.show()
            action = human_policy(observation)  # User-defined policy function
            print(action)
            observation, reward, terminated, info = env.step(action)
            cumulative_reward+=reward
            steps+=1
            if terminated:
                break
        print(f'Cumulative Reward ~ {cumulative_reward}; TimeTaken ~ {steps}')
    env.close()

In [ ]:
best_env(number = 1)

In [ ]:
!python3 -W ignore ppo.py

In [ ]:
plot_ppo(suffix[0][-1])
run_best_env(1,suffix[0][-1])

In [ ]:
!python3 -W ignore ppo.py

In [ ]:
plot_ppo(suffix[1][-1])
run_best_env(1,suffix[1][-1])

## Run more difficult tasks.

In [ ]:
!python3 -W ignore ppo.py --model 0 --ease 0

In [ ]:
plot_ppo(suffix[0][0]) # Plot training set.

arr = []
with open(f'results/test_set{suffix[0][0]}.json', 'r') as file:
    arr=json.load(file)

run_best_env(arr[-1],suffix[0][0]) # See the performance on the most difficult number.

In [ ]:
!python3 -W ignore ppo.py --model 1 --ease 0

In [ ]:
plot_ppo(suffix[1][0]) # Plot training set.

arr = []
with open(f'results/test_set{suffix[0][0]}.json', 'r') as file:
    arr=json.load(file)

run_best_env(arr[-1],suffix[1][0]) # See the performance on the most difficult number.

In [ ]:
def sum_digits(no)->int:
    res=0
    while no!=0:
        m=no%10
        res+=m 
        no=no//10
    return res 
valid=[]
for i in range(1,100):
    if sum_digits(i)<=10:
        valid.append(i)
def compare(i1,i2):
    return sum_digits(i1) - sum_digits(i2)
m=int(len(valid)*0.8)
print(m)

In [ ]:
!python3 -W ignore ppo.py

In [11]:
import numpy as np 

def test(tr_size = 100):
    indx = [ i for i in range(tr_size)]
    mini_batch_size = 16
    np.random.shuffle(indx)
    lb = 0
    temp = []
    while lb < tr_size:
        rand_ids = indx[lb: min(lb+mini_batch_size, tr_size)]
        lb += mini_batch_size
        temp.extend(rand_ids)
        print(rand_ids)
    temp.sort()
    assert temp[-1] == tr_size - 1

test(5)

[0, 1, 3, 2, 4]
